#### We use the Dataset provided by the British Health Authoritites to find information about the Geolocation of each Tesco Store in the London Area. 
Other Ways seemed rather hard to get the Location data

In [1]:
import pandas as pd
import xml.etree.ElementTree as et
import urllib.request
import pandas_read_xml as pdrx
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

In [76]:
## Iterate over the LocalAuthorityCodes for the Greater London Area (501 - 533)
df_tesco_store_locations = pd.DataFrame()
df_top_four_locations = pd.DataFrame()

for x in range(501, 534):
    
    url =  "https://ratings.food.gov.uk/OpenDataFiles/FHRS{0}en-GB.xml".format(x)
    print("Loading Data for Municipality ", x, "From URL: ", url)
    
    # Load the DataSet directly from the XML File (Easily do able because Quasi Relational)
    df = pdrx.read_xml(url, ['FHRSEstablishment', 'EstablishmentCollection'])
    
    # Parsing the XML
    df["BusinessName"] = df["EstablishmentDetail"].apply(lambda x : x["BusinessName"])
    df["GeoCode"] = df["EstablishmentDetail"].apply(lambda x : x["Geocode"])
    df["Longitude"] = df["GeoCode"].apply(lambda x : x["Longitude"] if x is not None else np.nan)
    df["Latitude"] = df["GeoCode"].apply(lambda x : x["Latitude"] if x is not None else np.nan)
    df["LocalAuthorityName"] = df["EstablishmentDetail"].apply(lambda x : x["LocalAuthorityName"])

    df = df[["BusinessName", "Longitude", "Latitude", "LocalAuthorityName"]]
    
    df_all = df_all.append(df)
    
    # Checking for the TESCOS (and removing Tesco Subbrands (i.e Metro Stops) and Subtenants )
    df_asda = df[df["BusinessName"].apply(lambda x : True if x in ['Asda', 'Asda Stores', 'Asda supermarket'] else False)]
    df_sal = df[df["BusinessName"].apply(lambda x : True if x in ["Sainsbury's",'Sainsburys', "Sainsbury's Local", "Sainsbury's Supermarkets Ltd",'Sainsburys Supermarket',"Sainsbury's - Main Store", 'Sainsburys supermarkets ltd',"Sainsbury's Supermarket Ltd", "Sainsbury's Supermarkets",'Sainsburys supermarkets Ltd'] else False)]
    df_mor = df[df["BusinessName"].apply(lambda x : True if x in ["Morrisons Store","Morrisons Supermarket",'Morrisons','Morrisons (inc petrol station)'] else False)]
    df_tes = df[df["BusinessName"].apply(lambda x : True if x in ["Tesco", "Tesco Superstore", "Tesco Express", "Tesco Metro", "Tesco Supermarket",'Tesco Welling Superstore'] else False)]
    df_tesco_store_locations = df_tesco_store_locations.append(df_tes)
    
    df_top_four_locations = df_top_four_locations.append(df_tes)
    df_top_four_locations = df_top_four_locations.append(df_asda)
    df_top_four_locations = df_top_four_locations.append(df_mor)
    df_top_four_locations = df_top_four_locations.append(df_sal)
    

Loading Data for Municipality  501 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS501en-GB.xml
Loading Data for Municipality  502 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS502en-GB.xml
Loading Data for Municipality  503 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS503en-GB.xml
Loading Data for Municipality  504 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS504en-GB.xml
Loading Data for Municipality  505 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS505en-GB.xml
Loading Data for Municipality  506 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS506en-GB.xml
Loading Data for Municipality  507 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS507en-GB.xml
Loading Data for Municipality  508 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS508en-GB.xml
Loading Data for Municipality  509 From URL:  https://ratings.food.gov.uk/OpenDataFiles/FHRS509en-GB.xml
Loading Data for Municipality  510 From URL:  https://r

In [79]:
def rename_store(store_name):
    if "Asda" in store_name:
        return "Asda"
    elif "Sainsbury" in store_name:
        return "Sainsbury's"  
    elif "Morrisons" in store_name:
        return "Morrisons"
    elif "Tesco" in store_name:
        return "Tesco"
    
    return store_name
    


df_top_four_locations["BusinessName"] = df_top_four_locations["BusinessName"].apply(lambda x : rename_store(x)) 

In [83]:
df_tesco_store_locations.to_csv("Geo_Data/tesco_store_locations_greater_london.csv")
df_top_four_locations.to_csv("Geo_Data/top_four_store_locations_greater_london.csv")